# Compute the risk of flooding from a threshold extracted form a frequency analysis of observed data

In this notebook, we combine the forecasting abilities and the time series analysis capabilities in a single seamless process to estimate the flood risk of a probabilistic forecast. As an example, we first perform a frequency analysis on an observed time series, then estimate the streamflow associated to a 2-year return period. We then perform a climatological ESP forecast (to ensure repeatability, but a realtime forecast would work too!) and estimate the probability of flooding (exceeding the threshold) given the ensemble of members in the probabilistic forecast.

In [1]:
%matplotlib inline

import datetime as dt
import os
import tempfile
from pathlib import Path
from urllib.request import urlretrieve

import xarray as xr
from birdy import WPSClient
from matplotlib import pyplot as plt

from ravenpy.utilities.testdata import get_file

# Connect to the Ouranos finch server that handles the analysis of timeseries.
finch_url = os.environ.get(
    "FINCH_WPS_URL", "https://pavics.ouranos.ca/twitcher/ows/proxy/finch/wps"
)
finch = WPSClient(finch_url)

Perform the time series analysis on observed data for the catchment using the frequency analysis WPS capabilities.

In [ ]:
# Get the data that we will be using for the demonstration.
ts = get_file("raven-gr4j-cemaneige/Salmon-River-Near-Prince-George_meteo_daily.nc")

# Perform the frequency analysis for various return periods. We compute 2, 5, 10, 25, 50 and 100 year return
# periods, but later on we will only compare the forecasts to the 2 year return period.
resp_fa = finch.freq_analysis(
    ts, variable="qobs", mode="max", t=(2, 5, 10, 25, 50, 100), dist="gumbel_r"
)
out, log, meta = resp_fa.get(asobj=True)

In [ ]:
# Plot the results of the flows as a function of return period.
fig, ax = plt.subplots(1)
lines = out.q1maxannual.plot(ax=ax)

# Get 2-year return period from the frequency analysis
threshold = out.sel(return_period=2).q1maxannual.values
print(f"Threshold: {threshold:.1f}")

pt = ax.plot([2], [threshold], "ro")

ax.annotate(
    "Flow threshold, set at 2-year return period",
    (2, threshold),
    xytext=(25, 10),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->", connectionstyle="arc3"),
)

## Probabilistic forecast

In this example, we will perform an ensemble hydrological forecast and will then compute the probability of flooding given a flooding threshold. Start by building the model configuration as in the Tutorial Notebook 11:

In [ ]:
from ravenpy import Emulator, RavenWarning
from ravenpy.new_config import commands as rc
from ravenpy.new_config.emulators import GR4JCN
from ravenpy.utilities.forecasting import compute_forecast_flood_risk
from ravenpy.utilities.new_config import forecasting

# Choose the forecast date. Each forecast will start with the same day and month.
# For example, jan-05-2001 will compare the climatology using all jan-05ths from the dataset)
fdate = dt.datetime(2003, 4, 13)

# The dataset to use to get the forecast timeseries:
# ts = str(get_file("raven-gr4j-cemaneige/Salmon-River-Near-Prince-George_meteo_daily.nc"))
duration = 30  # Length in days of the climatological ESP forecast

# Define HRU to build the hydrological model
hru = {}
hru = dict(
    area=4250.6,
    elevation=843.0,
    latitude=54.4848,
    longitude=-123.3659,
    hru_type="land",
)

# Set alternative names for netCDF variables
alt_names = {
    "TEMP_MIN": "tmin",
    "TEMP_MAX": "tmax",
    "RAINFALL": "rain",
    "SNOWFALL": "snow",
}

# Data types to extract from netCDF
data_type = ["TEMP_MAX", "TEMP_MIN", "RAINFALL", "SNOWFALL"]

# Model configuration
model_config = GR4JCN(
    params=[0.529, -3.396, 407.29, 1.072, 16.9, 0.947],
    Gauge=rc.Gauge.from_nc(
        ts,
        data_type=data_type,
        alt_names=alt_names,
        extra={
            1: {
                "elevation": hru[
                    "elevation"
                ],  # No need for lat/lon as they are included in the netcdf file already
            }
        },
    ),
    HRUs=[hru],
    StartDate=fdate,
    Duration=duration,
    RunName="Probabilistic_flood_risk_NB",
    GlobalParameter={"AVG_ANNUAL_RUNOFF": 208.480},
)

Now that the configuration is ready, launch the ESP forecasting tool to generate an ensemble hydrological forecast:

In [ ]:
%%capture --no-display
# Adding this to avoid spamming warning messages for overwriting files.

# Launch the ESP forecasting method
ESP_sims = forecasting.climatology_esp(
    config=model_config,
    path="/tmp/Run_results_probabilistic_forecasts/",  # path to where we want to generate the rv files.
    overwrite=True,
)

# Show the results in an xarray dataset, ready to use:
ESP_sims.hydrograph

In [ ]:
# Plot the forecasts and the 2-year threshold previously estimated
fig, ax = plt.subplots(1)
ESP_sims.hydrograph.q_sim[:, :, 0].plot.line(
    ax=ax, hue="member", add_legend=False, color="gray", lw=0.5
)
t = ax.axhline(threshold, color="red")

In [ ]:
# Now compute the flood risk given the probabilistic forecast and the threshold associated to the 2-year return
# period.

threshold = out.sel(return_period=2).q1maxannual.values

# Run the flood forecast risk tool to extract the probability of exceedance in netcdf format and xarray Dataset format
flood_risk_file, flood_risk_data = compute_forecast_flood_risk(
    forecast=ESP_sims.hydrograph.q_sim,
    flood_level=threshold,
    path="/tmp/Run_results_probabilistic_forecasts",
)

# Extract the data and plot
fig, ax = plt.subplots(1)
l = flood_risk_data.exceedance_probability.plot()
ax.set_ylabel("Flood risk")

### Results analysis
We can see from the above figure that there is no risk of exceeding the 2-year return period for the selected dates of the forecast.